# API Client Usage Tutorial

Author: galafis\nDate: 2025-09-30\nObjective: Demonstrate safe, end-to-end usage of the API client (auth, requests, and handling).

## Overview and prerequisites

- Python 3.9+
- Dependencies: requests, python-dotenv (optional)
- Configure `.env` locally (never commit secrets)

## Setup (env vars, API keys)

Example `.env` (local only):

API_BASE_URL=https://api.example.com
API_KEY=replace-with-local-key

This notebook avoids printing secrets.

In [ ]:
# Load configuration
import os
try:
    from dotenv import load_dotenv
    load_dotenv()
except Exception:
    pass
API_BASE_URL = os.getenv('API_BASE_URL', 'https://api.example.com')
API_KEY = os.getenv('API_KEY', 'demo-key')
assert API_BASE_URL and API_KEY
print('Config loaded. Base URL OK; API key present (hidden).')

## Minimal client

Wraps headers, base URL, and basic methods with simple error handling.

In [ ]:
import requests
from typing import Optional, Dict, Any

class RTMClient:
    def __init__(self, base_url: str, api_key: str):
        self.base_url = base_url.rstrip('/')
        self.session = requests.Session()
        self.session.headers.update({
            'Authorization': f'Bearer {api_key}',
            'Content-Type': 'application/json',
            'Accept': 'application/json'
        })

    def _url(self, path: str) -> str:
        return f"{self.base_url}/{path.lstrip('/')}"

    def get(self, path: str, params: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
        r = self.session.get(self._url(path), params=params, timeout=30)
        r.raise_for_status()
        return r.json()

    def post(self, path: str, json: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
        r = self.session.post(self._url(path), json=json, timeout=30)
        r.raise_for_status()
        return r.json()

client = RTMClient(API_BASE_URL, API_KEY)
print('Client initialized.')

## Step-by-step walkthrough

Endpoints (synthetic placeholders, replace with project routes):
- /v1/health
- /v1/quotes
- /v1/search

In [ ]:
# Health check
try:
    health = client.get('/v1/health')
    print('Health:', health)
except Exception as e:
    print('Health check failed (expected with demo URL):', e)

In [ ]:
# Quotes with fallback to synthetic data
import pandas as pd
symbols = ['AAPL', 'MSFT', 'GOOG']
payload = {'symbols': symbols}
try:
    quotes = client.post('/v1/quotes', json=payload)
    df_quotes = pd.DataFrame(quotes.get('data', []))
    display(df_quotes.head())
except Exception as e:
    print('Quotes request failed (expected with demo URL):', e)
    df_quotes = pd.DataFrame([
        {'symbol': 'AAPL', 'price': 175.23, 'ts': '2025-09-30T00:00:00Z'},
        {'symbol': 'MSFT', 'price': 402.10, 'ts': '2025-09-30T00:00:00Z'},
        {'symbol': 'GOOG', 'price': 142.55, 'ts': '2025-09-30T00:00:00Z'},
    ])
    display(df_quotes)

In [ ]:
# Search with synthetic fallback
query = 'Apple earnings'
params = {'q': query, 'limit': 5}
try:
    results = client.get('/v1/search', params=params)
    for i, item in enumerate(results.get('items', []), 1):
        print(f"{i}. {item.get('title','N/A')} -> {item.get('url','')}")
except Exception as e:
    print('Search request failed (expected with demo URL):', e)
    items = [
        {'title': 'Apple posts quarterly earnings', 'url': 'https://example.com/aapl-earnings'},
        {'title': 'Analyst review on AAPL', 'url': 'https://example.com/aapl-analyst'}
    ]
    for i, item in enumerate(items, 1):
        print(f"{i}. {item['title']} -> {item['url']}")

## Validation / expected outputs

- Health ok in real env
- Quotes -> JSON list normalized into DataFrame
- Search -> items with titles/URLs
Synthetic fallbacks ensure runnable example without real API.

## Troubleshooting and tips

- Check env vars and network
- Handle HTTP errors with status and body
- Mask secrets in logs
- Parameterize workloads (env/papermill)
- Extract helpers to notebooks/utils/